In [140]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset

In [141]:
data = pd.read_csv("29Y_IQR.csv")
spectra = data.iloc[:, :-1]
label = data.iloc[:, -1]

data_2 = pd.read_csv("PAO & MSSA.csv")
spectra_2 = data_2.iloc[:, :-1]

In [142]:
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(label.values.reshape(-1,1))

X_train_tensor = torch.tensor(spectra.values, dtype = torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype = torch.float32)

c:\Users\elfo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [143]:
dataset= TensorDataset(X_train_tensor, y_train_tensor)
loader = DataLoader(dataset, batch_size = 128, shuffle = True)

In [144]:
class RamanNet(nn.Module):
    def __init__(self):
        super(RamanNet, self).__init__()
        self.fc1 = nn.Linear(spectra.shape[1], 256)  # Adjust the input features
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 6)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  # Sigmoid for binary classification on each label
        return x
    
model = RamanNet()

In [145]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

epochs = 50

for epoch in range(epochs):
    for spectra, labels in loader:
        optimizer.zero_grad()
        outputs = model(spectra)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.48574671149253845
Epoch 2, Loss: 0.47007179260253906
Epoch 3, Loss: 0.45333731174468994
Epoch 4, Loss: 0.4366679787635803
Epoch 5, Loss: 0.402376651763916
Epoch 6, Loss: 0.36849677562713623
Epoch 7, Loss: 0.32795342803001404
Epoch 8, Loss: 0.31524839997291565
Epoch 9, Loss: 0.2967030704021454
Epoch 10, Loss: 0.31296488642692566
Epoch 11, Loss: 0.30840229988098145
Epoch 12, Loss: 0.2965785562992096
Epoch 13, Loss: 0.26814770698547363
Epoch 14, Loss: 0.25259578227996826
Epoch 15, Loss: 0.24344433844089508
Epoch 16, Loss: 0.27961722016334534
Epoch 17, Loss: 0.23685815930366516
Epoch 18, Loss: 0.2588938772678375
Epoch 19, Loss: 0.23645901679992676
Epoch 20, Loss: 0.23789532482624054
Epoch 21, Loss: 0.2146650105714798
Epoch 22, Loss: 0.2075350135564804
Epoch 23, Loss: 0.21639806032180786
Epoch 24, Loss: 0.17372386157512665
Epoch 25, Loss: 0.19815713167190552
Epoch 26, Loss: 0.16410210728645325
Epoch 27, Loss: 0.16384470462799072
Epoch 28, Loss: 0.17432080209255219
Epoch 29,

In [146]:
test_tensor = torch.tensor(spectra_2.values, dtype = torch.float32)

predict = model(test_tensor)
print(predict)

tensor([[0.0527, 0.0082, 0.0621, 0.0254, 0.3303, 0.0076],
        [0.1491, 0.0082, 0.0533, 0.0410, 0.2088, 0.0070]],
       grad_fn=<SigmoidBackward0>)
